In [1]:
import pandas as pd
import numpy as np
import os
import warnings
from sklearn import preprocessing

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from sklearn.utils.class_weight import compute_class_weight
from sklearn import preprocessing

import torch
import torch.nn as nn
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.autograd import Variable

desired_frames = 1
desired_features = 2048

In [2]:
labels = pd.read_csv("/Users/yoshithaakunuri/Documents/CSCI535/Project/Final/data/scene_labels.csv")

In [3]:
labels.head(5)

,SCENE,KEY,SPEAKER,SHOW,Sarcasm,Sarcasm_Type
0,1_10004,1_10004_u,SHELDON,BBT,0.0,NONE
1,1_10009,1_10009_u,PENNY,BBT,0.0,NONE
2,1_1001,1_1001_u,RAJ,BBT,0.0,NONE
3,1_1003,1_1003_u,HOWARD,BBT,1.0,PRO
4,1_10190,1_10190_u,SHELDON,BBT,0.0,NONE


In [4]:
scenes = list(pd.unique(labels["SCENE"]))

In [5]:
len(scenes)

1202

In [6]:
parent_dir = "/Users/yoshithaakunuri/Documents/CSCI535/Project/Final/data/features"
visual_features = {}
context_visual_features = {}
for i in range(len(scenes)):
    # Visual Features
    try:
        vf = np.load(os.path.join(parent_dir, "utterances_final", "resnet_pool5_" + scenes[i] + ".npy"))
        #Global average pooling
        vf_p = np.apply_over_axes(np.mean, vf, [2, 3]) 
        vf_p = np.reshape(vf_p, (vf_p.shape[0],2048))
        if vf_p.shape[0] != 15:
            vf_p = np.pad(vf_p, [(0, 15 - vf_p.shape[0]), (0, 0)], mode='constant')
    except:
        vf_p = np.zeros((15, 2048))
    
    # Visual Features - Context
    try:
        c_vf = np.load(os.path.join(parent_dir, "context_final", "resnet_pool5_" + scenes[i] + "_c.npy"))
        #Global average pooling
        c_vf_p = np.apply_over_axes(np.mean, c_vf, [2, 3]) 
        c_vf_p = np.reshape(c_vf_p, (c_vf_p.shape[0],2048))
        if c_vf_p.shape[0] != 15:
            c_vf_p = np.pad(c_vf_p, [(0, 15 - c_vf_p.shape[0]), (0, 0)], mode='constant')
    except:
        c_vf_p = np.zeros((15, 2048))
    
    visual_features[scenes[i]] = vf_p
    context_visual_features[scenes[i]] = c_vf_p

In [7]:
def get_model_data(context_video_features, video_features):
    model_data = pd.DataFrame(columns=['context_video_feature', 'video_feature','sarcasm','sarcasm_type', 'speaker'])
    for index, row in labels.iterrows():
#         audio_key = row["SCENE"] + "_u.wav"
        model_data = model_data.append({'context_video_feature': context_video_features[row["SCENE"]],
                                    'video_feature': video_features[row["SCENE"]],
                                    'sarcasm' : row["Sarcasm"],
                                    'sarcasm_type' : row["Sarcasm_Type"],
                                    'speaker' : row["SPEAKER"]},
                                  ignore_index=True)
    return model_data

In [8]:
def get_train_test_split(model_data, x_columns, y_column, stratify_column):
    X_train, X_test, Y_train, Y_test = train_test_split(
        model_data[x_columns],
        model_data[y_column],
        train_size=0.8, 
        test_size=0.2, 
        random_state=42, 
        shuffle=True,
        stratify=model_data[stratify_column])
    
    print("Train: ",X_train.shape, Y_train.shape,
      "Test: ",(X_test.shape, Y_test.shape))
    train_data = pd.merge(X_train, Y_train, left_index=True, right_index=True)
    test_data = pd.merge(X_test, Y_test, left_index=True, right_index=True)
    return train_data, test_data

In [9]:
class RNNTensorDataset(Dataset):
    def __init__(self, dataframe, speaker):
        self.data = dataframe
        self.speaker = speaker

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        if self.speaker:
            features = self.data.loc[index, 'padded_video_feature']
            ctxt_features = self.data.loc[index, 'padded_context_video_feature']
            final_features = np.vstack((ctxt_features, features))
            a=np.empty((30,1))
            a.fill(self.data.loc[index, 'speaker_encode'])
            final_features = np.hstack((final_features, a))
            label = self.data.loc[index, 'sarcasm']
            return torch.from_numpy(final_features).float(), label
        else:
            features = self.data.loc[index, 'padded_video_feature']
            ctxt_features = self.data.loc[index, 'padded_context_video_feature']
            final_features = np.vstack((ctxt_features, features))
            label = self.data.loc[index, 'sarcasm']
            return torch.from_numpy(final_features).float(), label
    
    def __getindexlist__(self):
        return list(self.data.index.values)
    
class RNNetSD(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, layers):
        super(RNNetSD, self).__init__()
        self.hidden_dim = hidden_dim
        self.layers = layers
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True, nonlinearity="relu")
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = Variable(torch.zeros(self.layers, x.size(0), self.hidden_dim))
        out, hn = self.rnn(x, h0)
        out = F.softmax(self.fc(out[:, -1, :]))
        return out
    
class RNNetSID(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, layers):
        super(RNNetSID, self).__init__()
        self.hidden_dim = hidden_dim
        self.layers = layers
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True, nonlinearity="relu")
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = Variable(torch.zeros(self.layers, x.size(0), self.hidden_dim))
        out, hn = self.rnn(x, h0)
        out = F.softmax(self.fc(out[:, -1, :]))
        return out
    
def evaluateRNN(rnn, review, size):
    output = rnn(review)
    return output

def categoryFromOutput(output):
    top_n, top_i = torch.max(output,dim=1)
    return top_i

def test_accuracy(rnn, loader, size):
    actuals = []
    predictions = []
    for data, target in loader:
        output = evaluateRNN(rnn, data, size)
        prediction_index = categoryFromOutput(output)
        predictions = prediction_index.tolist()
        actuals = target.tolist()
    return predictions, actuals

In [10]:
warnings.filterwarnings("ignore")

model_data = get_model_data(context_visual_features, visual_features)
# Label Encode Speaker
le = preprocessing.LabelEncoder()
model_data['speaker_encode'] = le.fit_transform(model_data['speaker'])
model_data.head(5)

,context_video_feature,video_feature,sarcasm,sarcasm_type,speaker,speaker_encode
0,"[[0.30626756, 0.66872513, 0.28155187, 0.312841...","[[0.13835047, 0.2704592, 0.44648886, 0.1415337...",0.0,NONE,SHELDON,25
1,"[[0.17222668, 0.7350801, 0.30408216, 0.3297085...","[[0.46479157, 0.1813915, 0.22123067, 0.5245148...",0.0,NONE,PENNY,15
2,"[[0.048892517, 0.23698963, 0.4193368, 0.341263...","[[0.253619, 0.25664786, 0.6646118, 0.4821793, ...",0.0,NONE,RAJ,21
3,"[[0.5546928, 0.36914897, 0.8243424, 0.18547274...","[[0.55624646, 0.16990338, 0.62457716, 0.209021...",1.0,PRO,HOWARD,7
4,"[[0.73847526, 1.0405933, 0.11508263, 0.9313859...","[[0.6140023, 0.4846397, 0.79425097, 0.13518682...",0.0,NONE,SHELDON,25


In [11]:
train_data, test_data = get_train_test_split(model_data, ['context_video_feature', 'video_feature', 'speaker_encode'], 'sarcasm', 'sarcasm_type')
rnn_train = train_data.copy()
rnn_test = test_data.copy()
rnn_train.reset_index(drop=True, inplace = True)
rnn_test.reset_index(drop=True, inplace = True)

Train:  (961, 3) (961,) Test:  ((241, 3), (241,))


In [12]:
desired_length = 15
desired_length_ctxt = 15

rnn_train['padded_video_feature'] = rnn_train.loc[:, 'video_feature']
rnn_train['padded_context_video_feature'] = rnn_train.loc[:, 'context_video_feature']

rnn_test['padded_video_feature'] = rnn_test.loc[:, 'video_feature']
rnn_test['padded_context_video_feature'] = rnn_test.loc[:, 'video_feature']

rnn_train["sarcasm"] = rnn_train["sarcasm"].astype('int').to_numpy()
rnn_test["sarcasm"] = rnn_test["sarcasm"].astype('int').to_numpy()

In [13]:
EMBEDDING_DIM_sid = 2048
EMBEDDING_DIM_sd = 2049
HIDDEN_DIM = 20
OUTPUT_DIM = 2
layers = 2
criterion = nn.NLLLoss()

### Speaker Independent

In [14]:
rnn_train_tensor = RNNTensorDataset(rnn_train[['padded_context_video_feature', 'padded_video_feature', 'sarcasm']], False)
rnn_test_tensor = RNNTensorDataset(rnn_test[['padded_context_video_feature', 'padded_video_feature', 'sarcasm']], False)

num_of_workers = 0
batch_size = 31
valid_size = 0.1

train_indices = list(range(len(rnn_train_tensor)))
np.random.shuffle(train_indices)

test_indices = list(range(len(rnn_test_tensor)))
np.random.shuffle(test_indices)

train_loader = torch.utils.data.DataLoader(
    rnn_train_tensor, 
    batch_size=batch_size, 
    sampler=SubsetRandomSampler(train_indices)
)

test_loader = torch.utils.data.DataLoader(
    rnn_test_tensor, 
    batch_size=batch_size, 
    sampler=SubsetRandomSampler(test_indices)
)

test_loader_epoch = torch.utils.data.DataLoader(
    rnn_test_tensor, batch_size=rnn_test_tensor.__len__())

In [15]:
rnn = RNNetSID(EMBEDDING_DIM_sid, HIDDEN_DIM, OUTPUT_DIM, layers)
print(rnn)

optimizer = torch.optim.Adam(rnn.parameters(), lr=0.001)


RNNetSID(
  (rnn): RNN(2048, 20, num_layers=2, batch_first=True)
  (fc): Linear(in_features=20, out_features=2, bias=True)
)


In [16]:
n_epochs = 101
    
test_min_loss = np.inf

for epoch in range(n_epochs):
    torch.manual_seed(42)
    train_loss = 0.0
    test_loss = 0.0
    rnn.train()
    for data, target in train_loader:
        optimizer.zero_grad()
        output = rnn(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)

    rnn.eval()
    for data, target in test_loader:
        if data.shape[1] < 31:
            continue
        output = rnn(data)
        loss = criterion(output, target)
        test_loss += loss.item()*data.size(0)

    train_loss = train_loss / len(train_loader.dataset)
    test_loss = test_loss / len(test_loader.dataset)

    if(epoch%20 == 0):
        print("Epoch: " + str(epoch))
        predictions, actuals = test_accuracy(rnn, test_loader_epoch, rnn_test_tensor.__len__())
        print(classification_report(actuals, predictions, digits=4))

Epoch: 0
              precision    recall  f1-score   support

           0     0.5311    0.9250    0.6748       120
           1     0.7188    0.1901    0.3007       121

    accuracy                         0.5560       241
   macro avg     0.6249    0.5575    0.4877       241
weighted avg     0.6253    0.5560    0.4869       241

Epoch: 20
              precision    recall  f1-score   support

           0     0.6121    0.5917    0.6017       120
           1     0.6080    0.6281    0.6179       121

    accuracy                         0.6100       241
   macro avg     0.6100    0.6099    0.6098       241
weighted avg     0.6100    0.6100    0.6098       241

Epoch: 40
              precision    recall  f1-score   support

           0     0.4979    1.0000    0.6648       120
           1     0.0000    0.0000    0.0000       121

    accuracy                         0.4979       241
   macro avg     0.2490    0.5000    0.3324       241
weighted avg     0.2479    0.4979    0.3310  

### Speaker Dependent

In [17]:
rnn_train_tensor = RNNTensorDataset(rnn_train[['padded_context_video_feature', 'padded_video_feature', 'speaker_encode','sarcasm']], True)
rnn_test_tensor = RNNTensorDataset(rnn_test[['padded_context_video_feature', 'padded_video_feature', 'speaker_encode', 'sarcasm']], True)

num_of_workers = 0
batch_size = 31
valid_size = 0.1

train_indices = list(range(len(rnn_train_tensor)))
np.random.shuffle(train_indices)

test_indices = list(range(len(rnn_test_tensor)))
np.random.shuffle(test_indices)

train_loader = torch.utils.data.DataLoader(
    rnn_train_tensor, 
    batch_size=batch_size, 
    sampler=SubsetRandomSampler(train_indices)
)

test_loader = torch.utils.data.DataLoader(
    rnn_test_tensor, 
    batch_size=batch_size, 
    sampler=SubsetRandomSampler(test_indices)
)

test_loader_epoch = torch.utils.data.DataLoader(
    rnn_test_tensor, batch_size=rnn_test_tensor.__len__())

In [18]:
rnn = RNNetSD(EMBEDDING_DIM_sd, HIDDEN_DIM, OUTPUT_DIM, layers)
print(rnn)

optimizer = torch.optim.Adam(rnn.parameters(), lr=0.001)


RNNetSD(
  (rnn): RNN(2049, 20, num_layers=2, batch_first=True)
  (fc): Linear(in_features=20, out_features=2, bias=True)
)


In [19]:
n_epochs = 101
    
test_min_loss = np.inf

for epoch in range(n_epochs):
    torch.manual_seed(42)
    train_loss = 0.0
    test_loss = 0.0
    rnn.train()
    for data, target in train_loader:
        optimizer.zero_grad()
        output = rnn(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)

    rnn.eval()
    for data, target in test_loader:
        if data.shape[1] < 31:
            continue
        output = rnn(data)
        loss = criterion(output, target)
        test_loss += loss.item()*data.size(0)

    train_loss = train_loss / len(train_loader.dataset)
    test_loss = test_loss / len(test_loader.dataset)

    if(epoch%10 == 0):
        print("Epoch: " + str(epoch))
        predictions, actuals = test_accuracy(rnn, test_loader_epoch, rnn_test_tensor.__len__())
        print(classification_report(actuals, predictions, digits=4))

Epoch: 0
              precision    recall  f1-score   support

           0     0.6127    0.7250    0.6641       120
           1     0.6667    0.5455    0.6000       121

    accuracy                         0.6349       241
   macro avg     0.6397    0.6352    0.6321       241
weighted avg     0.6398    0.6349    0.6319       241

Epoch: 10
              precision    recall  f1-score   support

           0     0.7000    0.5833    0.6364       120
           1     0.6454    0.7521    0.6947       121

    accuracy                         0.6680       241
   macro avg     0.6727    0.6677    0.6655       241
weighted avg     0.6726    0.6680    0.6656       241

Epoch: 20
              precision    recall  f1-score   support

           0     0.7111    0.5333    0.6095       120
           1     0.6291    0.7851    0.6985       121

    accuracy                         0.6598       241
   macro avg     0.6701    0.6592    0.6540       241
weighted avg     0.6700    0.6598    0.6542  